# Collecting Reddit Data from r/WallStreetBets

## Part 1: Setup

### Install Packages

In [18]:
# Load environment variables from .env file
from dotenv import load_dotenv, dotenv_values
load_dotenv('.env')
config = dotenv_values()

In [19]:
# Import Packages
import praw, time, os, pyarrow
from IPython.display import display
from requests import Session
import pandas as pd
from IPython import get_ipython
from tqdm import tqdm 
import kagglehub, kaggle


# Get config from colab or other environment.
def is_colab():
    return get_ipython().__class__.__module__ == "google.colab._shell"

if is_colab():
    from google.colab import userdata
    config = {}
    config['CLIENT_SECRET'] = userdata.get('CLIENT_SECRET')
    config['CLIENT_ID'] = userdata.get('CLIENT_ID')
    config['NAME'] = userdata.get('NAME')
    config['REDIRECT_URI'] = userdata.get('REDIRECT_URI')
    config['USERNAME'] = userdata.get('USERNAME')
    config['PASSWORD'] = userdata.get('PASSWORD')

else:
    load_dotenv('.env')
    config = dotenv_values()


## Part 2: Collecting Submissions from Reddit

### Open Reddit Connection

In [20]:
# Create a custom session with a timeout
session = Session()
session.headers.update({'User-Agent': 'praw'})
session.timeout = 10  # Set a timeout of 10 seconds

# Login to Reddit using PRAW
reddit = praw.Reddit(
    client_id=config['CLIENT_ID'],
    client_secret=config['CLIENT_SECRET'],
    requestor_kwargs={"session": session},
    username=config['USERNAME'],
    password=config['PASSWORD'],
    user_agent="CS470 ML Project Access by u/GregorybLafayetteML"
)

# Add some peripheral config data
reddit.config.log_requests = 1
reddit.config.store_json_result = True

# Test the connection
try:
    username = reddit.user.me()
    print("Successfully logged in to Reddit!")
    print(f"Logged in as: u/{username}")
except Exception as e:
    print(f"Failed to log in: {e}")

Successfully logged in to Reddit!
Logged in as: u/GregorybLafayetteML


### Accessing Reddit Data

To access reddit posts, we'll need send a request with the number of post we want to get. The following example finds the top 10 hottest posts on the u/wallstreetbets subreddit. We'll show the post title, score, flair, and URL.

In [21]:
top_posts = reddit.subreddit('wallstreetbets').hot(limit=10)
print("Top 10 hot posts from r/wallstreetbets:")
for post in top_posts:
    print(f"Title: {post.title}, Score: {post.score}, Flair: {post.link_flair_text}, URL: {post.url}")

Top 10 hot posts from r/wallstreetbets:
Title: Weekend Discussion Thread for the Weekend of May 09, 2025, Score: 135, Flair: Weekend Discussion, URL: https://www.reddit.com/r/wallstreetbets/comments/1kirpy3/weekend_discussion_thread_for_the_weekend_of_may/
Title: Weekly Earnings Thread 5/12 - 5/16, Score: 106, Flair: Earnings Thread, URL: https://i.redd.it/4z7v8xc4srze1.jpeg
Title: I turned $55 into $1544 from Microsoft earnings 🌝, Score: 223, Flair: Gain, URL: https://i.redd.it/1w1dv0gi500f1.jpeg
Title: US and China end trade talks for the night,  will continue talks on Sunday., Score: 149, Flair: Discussion, URL: https://www.reddit.com/r/wallstreetbets/comments/1kji1re/us_and_china_end_trade_talks_for_the_night_will/
Title: Withdrew my 401k from my days in the army and put 2k on calls for. . . How screwed am I Monday?, Score: 127, Flair: YOLO, URL: https://www.reuters.com/world/us-china-trade-talks-geneva-end-night-continue-sunday-source-says-2025-05-10/
Title: Green smoke coming out

For this project, we'll need far more than ten posts at a time. The reddit API will limit our access to 100 posts at a time. Fortunately, the api uses a ListingGenerator which allows us to access our metered connection in sequential blocks. The following example shows how we can utilize this behavior, grabbing blocks of 100 posts at a time. In our example, we'll grab blocks of posts until we reach 5000 posts or our access times out. Notice that the procedure ends early with around 750-800 posts collected. The results are sparce, because our connection either timed out or was metered down by reddit. The latter option is more likely.

In [22]:
# Access the subreddit
subreddit = reddit.subreddit("wallstreetbets")

# Initialize variables
batch_size = 50 # Number of posts per batch
total_posts = 5000  # Total number of posts to fetch
all_posts = []  # To store all the retrieved posts
after = None  # To keep track of the last post for pagination

# Fetch posts in batches
while len(all_posts) < total_posts:
    # Fetch the next batch of posts
    submissions = subreddit.new(limit=batch_size, params={"after": after})

    batch_posts = []
    for submission in tqdm(submissions, desc="Storing batch of posts", unit="post"):
        batch_posts.append(submission)

        # Update the `after` variable with the last submission's fullname
        after = submission.fullname

    # Add the batch to the main list
    all_posts.extend(batch_posts)

    # Exit loop if no more posts are available
    if not batch_posts:
        print("No more posts to fetch.")
        break

    # Optional delay to avoid rate limits
    time.sleep(1)  # Adjust the delay as necessary

# Process the data (example: print the total number of posts fetched)
print(f"Fetched {len(all_posts)} posts in total.")

Storing batch of posts: 50post [00:00, 61.17post/s]
Storing batch of posts: 50post [00:00, 79.75post/s]
Storing batch of posts: 50post [00:00, 67.59post/s]
Storing batch of posts: 50post [00:00, 82.55post/s]
Storing batch of posts: 50post [00:00, 76.38post/s]
Storing batch of posts: 50post [00:00, 73.66post/s]
Storing batch of posts: 50post [00:00, 75.55post/s]
Storing batch of posts: 50post [00:00, 66.98post/s]
Storing batch of posts: 50post [00:00, 77.49post/s]
Storing batch of posts: 50post [00:00, 73.74post/s]
Storing batch of posts: 50post [00:01, 27.38post/s]
Storing batch of posts: 50post [00:00, 61.65post/s]
Storing batch of posts: 50post [00:00, 66.31post/s]
Storing batch of posts: 50post [00:00, 68.41post/s]
Storing batch of posts: 50post [00:00, 56.21post/s]
Storing batch of posts: 50post [00:00, 84.70post/s]
Storing batch of posts: 50post [00:00, 67.74post/s]
Storing batch of posts: 4post [00:00, 22.35post/s]
Storing batch of posts: 0post [00:00, ?post/s]

No more posts to fetch.
Fetched 854 posts in total.


Now that we have collected a large portion of posts/submssions, we'll parse the results and construct a dataframe with this data. We're going to collect more fields from this data than we might need right now, avoiding data limitations in the future.

In [23]:
# Parse are submission objects that we collected.
fields = ('title',
          'created_utc',
          'id',
          'is_original_content',
          'link_flair_text',
          'locked',
          'name',
          'num_comments',
          'over_18',
          'permalink',
          'selftext',
          'spoiler',
          'upvote_ratio')
list_of_submissions = []

# Parse each submission into a dictionary of the lised fields.
for submission in all_posts:
    full = vars(submission)
    sub_dict = {field:full[field] for field in fields}
    list_of_submissions.append(sub_dict)

# Create a python dataframe of these submissions.
collected_data = pd.DataFrame.from_records(list_of_submissions)

# Display the dataframe.
display(collected_data)

,title,created_utc,id,is_original_content,link_flair_text,locked,name,num_comments,over_18,permalink,selftext,spoiler,upvote_ratio
0,Tesla tells Model Y and Cybertruck workers to ...,1.746910e+09,1kjk032,False,News,False,t3_1kjk032,4,False,/r/wallstreetbets/comments/1kjk032/tesla_tells...,I can’t help but think this is a serious bull ...,False,1.00
1,The key people in the USA-China meeting are no...,1.746910e+09,1kjjxyw,False,Discussion,False,t3_1kjjxyw,9,False,/r/wallstreetbets/comments/1kjjxyw/the_key_peo...,I noticed that we sent our head of finance Sco...,False,0.78
2,Cut losses or hodl? Anyone else on the same boat,1.746909e+09,1kjjmgf,False,Loss,False,t3_1kjjmgf,8,False,/r/wallstreetbets/comments/1kjjmgf/cut_losses_...,Perfect timing buying $BULL calls before the o...,False,1.00
3,Withdrew my 401k from my days in the army and ...,1.746907e+09,1kjitpt,False,YOLO,False,t3_1kjitpt,162,False,/r/wallstreetbets/comments/1kjitpt/withdrew_my...,I have 13 calls on uvxy between 32-40. I pulle...,False,0.78
4,"US and China end trade talks for the night, w...",1.746905e+09,1kji1re,False,Discussion,False,t3_1kji1re,87,False,/r/wallstreetbets/comments/1kji1re/us_and_chin...,Trade talks to continue tomorrow.\n\nhttps://w...,False,0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,"TSMC first-quarter profit tops estimates, risi...",1.744869e+09,1k15iv3,False,News,False,t3_1k15iv3,39,False,/r/wallstreetbets/comments/1k15iv3/tsmc_firstq...,,False,0.97
850,"Mortgage rates soar, prompting home buyers to ...",1.744867e+09,1k156sh,False,News,False,t3_1k156sh,351,False,/r/wallstreetbets/comments/1k156sh/mortgage_ra...,The Big Short Part 2: Electric Spoon,False,0.98
851,Long calls. Am i safe?,1.744867e+09,1k152pc,False,Discussion,False,t3_1k152pc,53,False,/r/wallstreetbets/comments/1k152pc/long_calls_...,Long calls. Am i safe?,False,0.86
852,DIS ber case.. more downside?,1.744867e+09,1k15220,False,Discussion,False,t3_1k15220,22,False,/r/wallstreetbets/comments/1k15220/dis_ber_cas...,"Aside from the recent Snow White debacle, here...",False,0.56


### Saving Reddit Data

In [24]:
# Save the collected data to parquet format
SUBMISSION_PARQUET_PATH = '../data/wallstreetbets-collection.parquet'

# Create a pyarrow schema for the data types.
submission_schema = pyarrow.schema([
    ('title', pyarrow.string()),
    ('created_utc', pyarrow.float64()),
    ('id', pyarrow.string()),
    ('is_original_content', pyarrow.bool_()),
    ('link_flair_text', pyarrow.string()),
    ('locked', pyarrow.bool_()),
    ('name', pyarrow.string()),
    ('num_comments', pyarrow.int64()),
    ('over_18', pyarrow.bool_()),
    ('permalink', pyarrow.string()),
    ('selftext', pyarrow.string()),
    ('spoiler', pyarrow.bool_()),
    ('upvote_ratio', pyarrow.float64()),
])

In [25]:
# If the parqet does not exist, create it.
if not os.path.exists(SUBMISSION_PARQUET_PATH):
    collected_data.to_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)

# If the data file already exist, merge new data with the existing one.
else:
    old_parquet = pd.read_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)
    new_parquet = pd.concat([old_parquet, collected_data])
    new_parquet = new_parquet.drop_duplicates(subset=['id','title','created_utc','name','permalink'], keep='last').reset_index(drop=True)
    new_parquet.to_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)

# Use the new collected data to get comment stuff.
SUBMISSION_PARQUET_PATH = '../data/wallstreetbets-collection.parquet'
submission_collection = pd.read_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)
display(submission_collection)

,title,created_utc,id,is_original_content,link_flair_text,locked,name,num_comments,over_18,permalink,selftext,spoiler,upvote_ratio
0,Nivea Along,1.744832e+09,1k0t4jk,False,YOLO,False,t3_1k0t4jk,5,False,/r/wallstreetbets/comments/1k0t4jk/nivea_along/,After -7% yesterday and -10% today,False,0.67
1,Powell to Volatile Stock Market: You’re on You...,1.744836e+09,1k0unbq,False,News,False,t3_1k0unbq,2,False,/r/wallstreetbets/comments/1k0unbq/powell_to_v...,,False,0.86
2,Made back the last Wendy’s paycheck I lost,1.744834e+09,1k0tv2y,False,Gain,False,t3_1k0tv2y,6,False,/r/wallstreetbets/comments/1k0tv2y/made_back_t...,,False,0.94
3,After market observation. When I finished buyi...,1.744833e+09,1k0tnqx,False,Gain,False,t3_1k0tnqx,8,False,/r/wallstreetbets/comments/1k0tnqx/after_marke...,https://preview.redd.it/41ilvj6f39ve1.png?widt...,False,0.72
4,Ominous,1.744833e+09,1k0thnd,False,Discussion,False,t3_1k0thnd,110,False,/r/wallstreetbets/comments/1k0thnd/ominous/,NVIDIA 2024 is starting to rhyme like Cisco 20...,False,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709,"TSMC first-quarter profit tops estimates, risi...",1.744869e+09,1k15iv3,False,News,False,t3_1k15iv3,39,False,/r/wallstreetbets/comments/1k15iv3/tsmc_firstq...,,False,0.97
1710,"Mortgage rates soar, prompting home buyers to ...",1.744867e+09,1k156sh,False,News,False,t3_1k156sh,351,False,/r/wallstreetbets/comments/1k156sh/mortgage_ra...,The Big Short Part 2: Electric Spoon,False,0.98
1711,Long calls. Am i safe?,1.744867e+09,1k152pc,False,Discussion,False,t3_1k152pc,53,False,/r/wallstreetbets/comments/1k152pc/long_calls_...,Long calls. Am i safe?,False,0.86
1712,DIS ber case.. more downside?,1.744867e+09,1k15220,False,Discussion,False,t3_1k15220,22,False,/r/wallstreetbets/comments/1k15220/dis_ber_cas...,"Aside from the recent Snow White debacle, here...",False,0.56


## Part 3: Collecting Comments from Reddit

### Creating a database of reddit threads

In [26]:
# Use the same methofology whih we used to collect submissions, but we'll add a parent submission id. and parent comment id.
# Since the comment section can be very deep, we'll limit comments to a breadth of 10.
# This may still be a lot more comments than we need for larger discussions.
def extract_comments_from_submission(submission_id: str):
    try:
        submission = reddit.submission(id=submission_id)
        submission.comments.replace_more(limit=10)  # Limit to 10 levels of comments
        comments = []

        for comment in submission.comments.list():
            if isinstance(comment, praw.models.MoreComments):
                continue

            # NOTE: It looks like the top comment may be a user report. We'll ignore if is has certain text.
            SKIPTEXT = '**User Report**'
            if SKIPTEXT in comment.body:
                continue

            # Append the comment data to the list
            comments.append({
                'parent_post_id': submission_id,
                'parent_comment_id': comment.parent_id,
                'comment_id': comment.id,
                'author': str(comment.author),
                'created_utc': comment.created_utc,
                'score': comment.score,
                'body': comment.body
            })

        return comments

    except Exception as e:
        # Get the HTTP error code if available
        if hasattr(e, 'response') and e.response is not None:
            error_code = e.response.status_code
            print(f"HTTP Error {error_code} while fetching comments for submission {submission_id}")
        else:
            error_code = None

        # Print the an erroor message and return nothing.
        print(f"Error fetching comments for submission {submission_id}: {e}")
        return []



In [27]:
# Show the results from one submission's comments
submission_id = submission_collection.iloc[0]['id']

# How many actual comments are there for this submission?
submission = reddit.submission(id=submission_id)
print(f"Submission ID: {submission_id}")
print(f"Title: {submission.title}")
print(f"Number of comments: {submission.num_comments}")

# Get the comments for the submission
results = extract_comments_from_submission(submission_id)

# Create a dataframe of the comments
comments_df = pd.DataFrame(results)

# Display the comments dataframe
display(comments_df)

Submission ID: 1k0t4jk
Title: Nivea Along
Number of comments: 6


,parent_post_id,parent_comment_id,comment_id,author,created_utc,score,body
0,1k0t4jk,t3_1k0t4jk,mngmxdi,chrissurra,1.744832e+09,5,You mean Neveah
1,1k0t4jk,t3_1k0t4jk,mngn956,Alert_Barber_3105,1.744832e+09,6,The skin cream?
2,1k0t4jk,t3_1k0t4jk,mngoka2,Reasonable_Roger,1.744832e+09,1,Psoriasis calls
3,1k0t4jk,t1_mngmxdi,mngnbpk,Own-Foundation3873,1.744832e+09,1,yesss sir
4,1k0t4jk,t1_mngn956,mngnj1n,Own-Foundation3873,1.744832e+09,1,pow till it creams


In [ ]:
# Collect the comments for all the submissions.
all_comments = []
for submission in tqdm(submission_collection['id'], desc="Fetching comments for all submissions", unit="submission"):    
    comments = extract_comments_from_submission(submission)
    all_comments.extend(comments)
    # time.sleep(1)  # Optional delay to avoid rate limits

# Create a python dataframe of these comments.
comments_df = pd.DataFrame.from_records(all_comments)
display(comments_df)

Fetching comments for all submissions:  59%|█████▉    | 1015/1714 [25:38<32:52,  2.82s/submission] 

HTTP Error 429 while fetching comments for submission 1kghdqf
Error fetching comments for submission 1kghdqf: received 429 HTTP response
HTTP Error 429 while fetching comments for submission 1kgggi0
Error fetching comments for submission 1kgggi0: received 429 HTTP response


Fetching comments for all submissions:  72%|███████▏  | 1241/1714 [30:29<29:32,  3.75s/submission]

In [ ]:
# Save the collected data to parquet format
COMMENT_PARQUET_PATH = '../data/wallstreetbets-comment-collection.parquet'

# Create a pyarrow schema for the comment data
comment_schema = pyarrow.schema([
    ('parent_post_id', pyarrow.string()),
    ('parent_comment_id', pyarrow.string()),
    ('comment_id', pyarrow.string()),
    ('author', pyarrow.string()),
    ('created_utc', pyarrow.float64()),
    ('score', pyarrow.int64()),
    ('body', pyarrow.string())
])

In [ ]:
# Write the comments to parquet file. If it exists, append to it.
if not os.path.exists(COMMENT_PARQUET_PATH):
    comments_df.to_parquet(COMMENT_PARQUET_PATH, engine='pyarrow', schema=comment_schema)
else:
    old_parquet = pd.read_parquet(COMMENT_PARQUET_PATH, engine='pyarrow', schema=comment_schema)
    new_parquet = pd.concat([old_parquet, comments_df])
    new_parquet = new_parquet.drop_duplicates(subset=['parent_post_id','parent_comment_id','author','created_utc'], keep='last').reset_index(drop=True)
    new_parquet.to_parquet(COMMENT_PARQUET_PATH, engine='pyarrow', schema=comment_schema)

# Part 4: Collecting More Data (w/ Kaggle)

### Read in the new reddit data from kaggle

In [ ]:
# Download path for the kaggle dataset.
KAGGLE_DATASET_PATH = '../data/kaggle-dataset'

kaggle.api.authenticate()
kaggle.api.dataset_download_files('gpreda/reddit-wallstreetsbets-posts', path='./data/', unzip=True)

In [ ]:
# Path to kaggle dataset.
KAGGLE_DATASET_RAW_PATH = '../data/reddit_wsb.csv'

# Read the kaggle dataset.
kaggle_df = pd.read_csv(KAGGLE_DATASET_RAW_PATH)

# Drop the timestamp column if it exists.
if 'timestamp' in kaggle_df.columns:
    kaggle_df = kaggle_df.drop(columns=['timestamp'])

# Enforce the schema.
kaggle_df = kaggle_df.astype({
    'title': 'string',
    'score': 'int64',
    'id': 'string',
    'url': 'string',
    'comms_num': 'int64',
    'created': 'float64',
    'body': 'string',
})

# Display the kaggle dataset.
display(kaggle_df)

In [ ]:
# Mapping to previous schema.
kaggle_mapping = {
    'title': 'title',
    'score': 'upvote_ratio',
    'id': 'id',
    'url': 'permalink',
    'comms_num': 'num_comments',
    'created': 'created_utc',
    'body': 'selftext'
}

# Rename the columns to match our schema.
kaggle_df.rename(columns=kaggle_mapping, inplace=True)

In [ ]:
# Path to kaggle dataset final.
KAGGLE_DATASET_FINAL_PATH = '../data/kaggle-reddit-wsb.parquet'

# Schema for the kaggle dataset.
kaggle_schema = pyarrow.schema([
    ('title', pyarrow.string()),
    ('upvote_ratio', pyarrow.int64()),
    ('id', pyarrow.string()),
    ('permalink', pyarrow.string()),
    ('num_comments', pyarrow.int64()),
    ('created_utc', pyarrow.float64()),
    ('selftext', pyarrow.string()),
])

# write the dataframe to parquet.
kaggle_df.to_parquet(KAGGLE_DATASET_FINAL_PATH, engine='pyarrow', schema=kaggle_schema)

### Merge the new data table with the old data.

In [ ]:
# Read in the datasets.
praw_dataset = pd.read_parquet(SUBMISSION_PARQUET_PATH, engine='pyarrow', schema=submission_schema)
kaggle_dataset = pd.read_parquet(KAGGLE_DATASET_FINAL_PATH, engine='pyarrow', schema=kaggle_schema)

# Create similar columns with only the columns of the kaggle dataset.
praw_dataset = praw_dataset[['title', 'upvote_ratio', 'id', 'permalink', 'num_comments', 'created_utc', 'selftext']]
kaggle_dataset = kaggle_dataset[['title', 'upvote_ratio', 'id', 'permalink', 'num_comments', 'created_utc', 'selftext']]

# Use the mapping to merge the two datasets.
merged_dataset = pd.concat([praw_dataset, kaggle_dataset], ignore_index=True)

# Remove duplicates based on the 'id' column.
merged_dataset = merged_dataset.drop_duplicates(subset=['id'], keep='last').reset_index(drop=True)

In [ ]:
# Creare a final schema for the merged dataset.
merged_schema = pyarrow.schema([
    ('title', pyarrow.string()),
    ('upvote_ratio', pyarrow.float64()),
    ('id', pyarrow.string()),
    ('permalink', pyarrow.string()),
    ('num_comments', pyarrow.int64()),
    ('created_utc', pyarrow.float64()),
    ('selftext', pyarrow.string())
])

# Save the merged dataset to parquet format.
MERGED_DATASET_PATH = '../data/merged-reddit-wsb.parquet'
merged_dataset.to_parquet(MERGED_DATASET_PATH, engine='pyarrow', schema=merged_schema)

In [ ]:
# Display properties of the merged dataset.
merged_dataset.info()
print("-" * 50)
print("Merged dataset saved to:", MERGED_DATASET_PATH)
print("Number of columns in merged dataset:", len(merged_dataset.columns))
print("Columns in merged dataset:", merged_dataset.columns.tolist())
print("-" * 50)
print("Number of rows in praw dataset:", len(praw_dataset))
print("Number of rows in kaggle dataset:", len(kaggle_dataset))
print("Number of rows in merged dataset:", len(merged_dataset))